In [ ]:
!pip install -q flwr[simulation] flwr_datasets[vision] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.7/364.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from os.path  import join
from collections import OrderedDict
import struct
from array import array
import numpy as np
from typing import List, Tuple, Dict, Any
import pandas as pd


# mount Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Mounted at /content/drive
Training on cpu using PyTorch 2.3.0+cu121 and Flower 1.9.0


## MNIST Data

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return torch.tensor(image, dtype=torch.float32).unsqueeze(0), torch.tensor(label, dtype=torch.int64)

class MnistDataloader(object):
    def __init__(self, training_images_filepath, training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        X_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        X_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (X_train, y_train), (X_test, y_test)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
input_path = '/content/drive/MyDrive/Colab Notebooks/data/mnist/data'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')

training_images_filepath

'/content/drive/MyDrive/Colab Notebooks/data/mnist/data/train-images-idx3-ubyte/train-images-idx3-ubyte'

In [ ]:
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(X_train, y_train), (X_test, y_test) = mnist_dataloader.load_data()

X_train, X_test = np.expand_dims(np.array(X_train), axis=1), np.expand_dims(np.array(X_test), axis=1)
y_train, y_test = np.array(y_train), np.array(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 1, 28, 28), (60000,), (10000, 1, 28, 28), (10000,))

In [17]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

## Load Data

In [18]:
BATCH_SIZE = 32

class CustomDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self, idx):
        x = torch.tensor(self.X_data[idx], dtype=torch.float32)
        y = torch.tensor(self.y_data[idx], dtype=torch.long)
        return x, y

# Splits a dataset into subsets for multiple clients.
def split_dataset(dataset, num_clients):
    subset_size = len(dataset) // num_clients
    subset_sizes = []
    for i in range(num_clients):
        if i + 1 == num_clients:
            subset_sizes.append(len(dataset) - i * subset_size)
        else:
            subset_sizes.append(subset_size)
    return subset_sizes

def load_data(num_clients):
    train_set = CustomDataset(X_train, y_train)
    test_set = CustomDataset(X_test, y_test)

    # Divide the data set randomly
    client_train_sets = random_split(train_set, split_dataset(train_set, num_clients))
    client_test_sets = random_split(test_set, split_dataset(test_set, num_clients))

    train_loaders = []
    test_loaders = []

    for i in range(num_clients):
        train_subset, test_subset = client_train_sets[i], client_test_sets[i]

        train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)

        train_loaders.append(train_loader)
        test_loaders.append(test_loader)

    num_examples = {"trainset" : len(train_set), "testset" : len(test_set)}

    return train_loaders, test_loaders, num_examples

## CNN

In [19]:
def train(net, trainloader, epochs):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    for _ in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()

def test(net, testloader):
    """Validate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    return loss, accuracy

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)        # 1 input channel (grayscale), 10 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(10, 32, 5)       # 10 input channels, 32 output channels, 5x5 kernel
        self.conv3 = nn.Conv2d(32, 64, 3)       # 32 input channels, 64 output channels, 3x3 kernel
        self.pool  = nn.MaxPool2d(2, 2)         # 2x2 pooling layer
        self.fc1   = nn.Linear(256, 128)        # Dense layer
        self.fc2   = nn.Linear(128, 64)         # Dense layer
        self.fc3   = nn.Linear(64, 10)          # 64 inputs, 10 outputs for the 10 classes
        # Steps:
            # Conv1: 28x28 -> 24x24 (28 - 5 + 1)
            # Pool:  24x24 -> 12x12 (24 / 2)
            # Conv2: 12x12 -> 8x8   (12 - 5 + 1)
            # Pool:  8x8   -> 4x4   (8 / 2)
            # Conv3: 4x4   -> 2x2   (4 - 3 + 1)
            # Flatn: 2x2   -> 1x256 (2 * 2 * 64)
            # Dense: 1x256 -> 1x128
            # Dense: 1x128 -> 1x64
            # Dense: 1x64  -> 1x10

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (12, 12, 10)
        x = self.pool(F.relu(self.conv2(x))) # (4, 4, 32)
        x = F.relu(self.conv3(x))            # (2, 2, 64)
        x = x.view(-1, 256)                  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [20]:
class MethodAClient(fl.client.NumPyClient):
    def __init__(self, net, train_loader, test_loader):
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.net = net

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train(self.net, self.train_loader, epochs=4)
        _, accuracy = test(self.net, self.test_loader)
        return self.get_parameters(config={}), num_examples["trainset"], {"accuracy": accuracy}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.test_loader)
        return float(loss), num_examples["testset"], {"accuracy": float(accuracy)}


In [21]:

class CustomFedMedian(fl.server.strategy.FedMedian):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.accuracies = []
        self.losses = []
        self.accuracy_df = pd.DataFrame()

    def aggregate_evaluate(
        self,
        rnd: int,
        results: List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.EvaluateRes]],
        failures: List[BaseException]
    ) -> Tuple[float, Dict[str, Any]]:

        round_accuracies = [result.metrics["accuracy"] for _, result in results]
        round_accuracy_dict = {f"Client {client_idx + 1}": accuracy for client_idx, accuracy in enumerate(round_accuracies)}
        round_df = pd.DataFrame(round_accuracy_dict, index=[rnd])
        accuracy = np.mean(round_accuracies)

        self.accuracies.append(accuracy)
        self.losses.append([result.loss for _, result in results])
        self.accuracy_df = pd.concat([self.accuracy_df, round_df])

        return super().aggregate_evaluate(rnd, results, failures)

    def plot_accuracies(self):
        plt.figure()
        plt.plot(range(1, len(self.accuracies) + 1), self.accuracies, marker='o')
        plt.title("Accuracy per Round")
        plt.xlabel("Round")
        plt.ylabel("Accuracy")
        plt.grid()
        plt.show()

    def plot_loss_per_client_per_round(self):
        num_rounds = len(self.losses)
        num_clients = len(self.losses[0])

        plt.figure()
        for client_idx in range(num_clients):
            plt.plot(range(1, num_rounds + 1), [self.losses[r][client_idx] for r in range(num_rounds)], marker='o', label=f'Client {client_idx + 1}')

        plt.title("Loss per Client per Round")
        plt.xlabel("Round")
        plt.ylabel("Loss")
        plt.legend()
        plt.grid()
        plt.show()

    def print_accuracy_matrix(self):
        """打印所有轮次和客户端的分类准确率矩阵"""
        print("Classification Accuracy Matrix:")
        print(self.accuracy_df)

In [22]:
def client_fn(cid: str) -> MethodAClient:
    train_loader = train_loaders[int(cid)]
    test_loader = test_loaders[int(cid)]
    net = Net().to(DEVICE)

    # print(f"cid: {int(cid)}")

    return MethodAClient(net, train_loader, test_loader).to_client()

## Start Simulation

### clients = 20, round = 50

In [23]:
# Load model and data
NUM_CLIENTS = 20
ROUND = 50

train_loaders, test_loaders, num_examples = load_data(NUM_CLIENTS)

train_loaders[0].dataset[0][0].shape

torch.Size([1, 28, 28])

In [24]:
# Load model and data
NUM_CLIENTS = 20
ROUND = 50

train_loaders, test_loaders, num_examples = load_data(NUM_CLIENTS)
strategy = CustomFedMedian()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=ROUND),
    strategy=strategy
)

strategy.plot_accuracies()
strategy.plot_loss_per_client_per_round()
strategy.print_accuracy_matrix()



INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout
/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-15 05:39:39,076	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 4013051904.0, 'memory': 8026103808.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parame

KeyboardInterrupt: 